In [2]:
import pandas as pd
import numpy as np
import psycopg2
import os

# Read Raw Data

In [7]:


def fetchDataFromDB(year,month):
   
    dbname = "fa71f"
    username = "fa71f"
    password = os.environ.get('CRED')
    host = "172.20.67.58"
    port = "5432"

    conn= psycopg2.connect(
        dbname=dbname,
        host=host,
        user=username,
        password=password,
        port=port
    )
    print('Connected')

    cur=conn.cursor()

    
    cur.execute(
    f"""
    SELECT hashed_user_id as uid,COUNT(DISTINCT date_trunc('day', sdk_ts)) AS m{month}
    FROM db_research_tamoco_by_year.tamoco_{year} 
    WHERE EXTRACT(MONTH FROM sdk_ts) = {month}
    """
    )

    result=cur.fetchall()

    cols=[desc[0] for desc in cur.description]
    
    cur.close()
    conn.close()


    
    return pd.DataFrame(result,columns=cols)

In [8]:
year=2019
raw_df=fetchDataFromDB(year,2)
#raw_df['month']=raw_df['datetime'].dt.month
#raw_df['dom']=raw_df['datetime'].dt.day
raw_df

Connected


,uid,active_days
0,002df443a1578976777acf0b7c9cb229,1
1,0075ee62bfebd7606c6c78f724cf9df2,1
2,010790b9b6d09839ec890c4952a935f5,1
3,0138543c57d577a4e7e7efc702ef365b,1
4,01690337947ce9eb4f8fbb2a601eca30,1
...,...,...
952,fd617723a5313fa53fe0b667233e83f0,1
953,fd8dabf52fe7221ab6ea739cd65f3076,1
954,fe1c199644de21f78a22653c7f1368e2,1
955,fef41d82fde4a489101815426c4716cc,1


# Claculate number of active days/month

In [5]:
active_df=pd.DataFrame(raw_df.groupby(['uid','month','dom'])['month'].apply(lambda x: 1))#(lambda x: np.where(len(x)>0,1,0) ))
active_df.rename(columns={'month':'active_status'},inplace=True)
active_df.reset_index(inplace=True)



In [6]:
active_df.sort_values(by=['uid','month','dom'],ascending=True,inplace=True)
active_df

,uid,month,dom,active_status
0,++E3j/UFIC0ZxfdM+8UeLw==,1,1,1
1,++E3j/UFIC0ZxfdM+8UeLw==,1,2,1
2,++E3j/UFIC0ZxfdM+8UeLw==,1,3,1
3,++E3j/UFIC0ZxfdM+8UeLw==,1,4,1
4,++E3j/UFIC0ZxfdM+8UeLw==,1,5,1
...,...,...,...,...
321736,zzyUhRMIlvM+gPDS6p5X0w==,2,1,1
321737,zzyUhRMIlvM+gPDS6p5X0w==,2,2,1
321738,zzyUhRMIlvM+gPDS6p5X0w==,2,3,1
321739,zzyUhRMIlvM+gPDS6p5X0w==,2,7,1


In [7]:
def getQuarter(x):
    months=x.values
    quarters=[]
    for month in months:
        if month>=1 and month<=3:
            quarters.append(1)
        elif month>=4 and month<=6:
            quarters.append(2)
        elif month>=7 and month<=9:
            quarters.append(3)
        elif month>=10 and month<=12:
            quarters.append(4)

    return quarters

In [8]:
active_df['quarter']=active_df.groupby(['uid','month','dom'])['month'].transform(getQuarter)
active_df

,uid,month,dom,active_status,quarter
0,++E3j/UFIC0ZxfdM+8UeLw==,1,1,1,1
1,++E3j/UFIC0ZxfdM+8UeLw==,1,2,1,1
2,++E3j/UFIC0ZxfdM+8UeLw==,1,3,1,1
3,++E3j/UFIC0ZxfdM+8UeLw==,1,4,1,1
4,++E3j/UFIC0ZxfdM+8UeLw==,1,5,1,1
...,...,...,...,...,...
321736,zzyUhRMIlvM+gPDS6p5X0w==,2,1,1,1
321737,zzyUhRMIlvM+gPDS6p5X0w==,2,2,1,1
321738,zzyUhRMIlvM+gPDS6p5X0w==,2,3,1,1
321739,zzyUhRMIlvM+gPDS6p5X0w==,2,7,1,1


In [9]:
tdf=pd.DataFrame(active_df.groupby(['uid','quarter']).size(),columns=['active_days']).reset_index()
tdf

,uid,quarter,active_days
0,++E3j/UFIC0ZxfdM+8UeLw==,1,87
1,++E3j/UFIC0ZxfdM+8UeLw==,2,91
2,++E3j/UFIC0ZxfdM+8UeLw==,3,85
3,++E3j/UFIC0ZxfdM+8UeLw==,4,87
4,++ZQpyJVkxuIYSKc8MWatw==,1,55
...,...,...,...
22049,zzf7KJrp2X1YZqVwPX9rdg==,2,1
22050,zzf7KJrp2X1YZqVwPX9rdg==,3,1
22051,zzkSogiotMTFKuNH0DpZvw==,2,1
22052,zzkhxJj/+X71C9ULcBWP/w==,1,7


In [10]:
tdf=tdf.pivot(index='uid',columns='quarter',values=['active_days']).add_prefix('quarter_').reset_index().fillna(0)
tdf.columns = [f'quarter_{col[1]}_active_days' if col[1] else col[0] for col in tdf.columns]
tdf.rename(columns={
    'quarter_quarter_1_active_days':'active_days_q1',
    'quarter_quarter_2_active_days':'active_days_q2',
    'quarter_quarter_3_active_days':'active_days_q3',
    'quarter_quarter_4_active_days':'active_days_q4'
},inplace=True)

tdf['total_active_days']=tdf['active_days_q1']+tdf['active_days_q2']+tdf['active_days_q3']+tdf['active_days_q4']

tdf

,uid,active_days_q1,active_days_q2,active_days_q3,active_days_q4,total_active_days
0,++E3j/UFIC0ZxfdM+8UeLw==,87.0,91.0,85.0,87.0,350.0
1,++ZQpyJVkxuIYSKc8MWatw==,55.0,91.0,30.0,0.0,176.0
2,++mNhOJRFKFMcMCUQ9jjjA==,2.0,0.0,0.0,0.0,2.0
3,+/9GKZn4Daa1UzhExi+GWg==,0.0,0.0,0.0,8.0,8.0
4,+/JFNu3kIP/9Xw1N8I7QDw==,0.0,0.0,8.0,0.0,8.0
...,...,...,...,...,...,...
15178,zyyrnuySmUgk/ou5mnSowg==,2.0,1.0,25.0,0.0,28.0
15179,zzf7KJrp2X1YZqVwPX9rdg==,0.0,1.0,1.0,0.0,2.0
15180,zzkSogiotMTFKuNH0DpZvw==,0.0,1.0,0.0,0.0,1.0
15181,zzkhxJj/+X71C9ULcBWP/w==,7.0,0.0,0.0,0.0,7.0


In [11]:
tdf

,uid,active_days_q1,active_days_q2,active_days_q3,active_days_q4,total_active_days
0,++E3j/UFIC0ZxfdM+8UeLw==,87.0,91.0,85.0,87.0,350.0
1,++ZQpyJVkxuIYSKc8MWatw==,55.0,91.0,30.0,0.0,176.0
2,++mNhOJRFKFMcMCUQ9jjjA==,2.0,0.0,0.0,0.0,2.0
3,+/9GKZn4Daa1UzhExi+GWg==,0.0,0.0,0.0,8.0,8.0
4,+/JFNu3kIP/9Xw1N8I7QDw==,0.0,0.0,8.0,0.0,8.0
...,...,...,...,...,...,...
15178,zyyrnuySmUgk/ou5mnSowg==,2.0,1.0,25.0,0.0,28.0
15179,zzf7KJrp2X1YZqVwPX9rdg==,0.0,1.0,1.0,0.0,2.0
15180,zzkSogiotMTFKuNH0DpZvw==,0.0,1.0,0.0,0.0,1.0
15181,zzkhxJj/+X71C9ULcBWP/w==,7.0,0.0,0.0,0.0,7.0


In [12]:
tdf.to_csv(f'D:\Mobile Device Data\OD_calculation_latest_work\HUQ_OD\\{year}\\active_days_stat_{year}.csv',index=False)